<a href="https://colab.research.google.com/github/saahiljain/CODES/blob/master/getting_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mounting google drive to colab to get the training data
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd /content/drive/My\ Drive

In [0]:
!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv

In [0]:
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv

In [0]:
!pip3 install awscli

In [0]:
!mkdir JPEGImages

In [0]:
!mkdir labels

In [0]:
import csv
import subprocess
import os

runMode = "train"
classes = ["Helmet"]//change here to get ambulance

with open('class-descriptions-boxable.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_list = {rows[1]:rows[0] for rows in reader}

subprocess.run([ '!mkdir', 'JPEGImages'],shell=True)

subprocess.run(['rm', '-rf', 'labels'],shell=True)
subprocess.run([ '!mkdir', 'labels'],shell=True)

for ind in range(0, len(classes)):
    
    className = classes[ind]
    print("Class " + str(ind) + " : " + className)

    commandStr = "grep " + dict_list[className] + " " + runMode + "-annotations-bbox.csv"
    print(commandStr)
    class_annotations = subprocess.run(commandStr.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
    class_annotations = class_annotations.splitlines()

    totalNumOfAnnotations = len(class_annotations)
    print("Total number of annotations : "+str(totalNumOfAnnotations))

    cnt = 0
    for line in class_annotations[0:totalNumOfAnnotations]:
        cnt = cnt + 1
        print("annotation count : val " + str(cnt))
        lineParts = line.split(',')
        print(lineParts)
        subprocess.run([ 'aws', 's3', '--no-sign-request', '--only-show-errors', 'cp', 's3://open-images-dataset/'+runMode+'/'+lineParts[0]+".jpg", 'JPEGImages/'+lineParts[0]+".jpg"])
        with open('labels/%s.txt'%(lineParts[0]),'a') as f:
            f.write(' '.join([str(ind),str((float(lineParts[5]) + float(lineParts[4]))/2), str((float(lineParts[7]) + float(lineParts[6]))/2), str(float(lineParts[5])-float(lineParts[4])),str(float(lineParts[7])-float(lineParts[6]))])+'\n')

print("done")
